In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import plotly.graph_objects as go
import sys
sys.path.append('../../assets/python/')
import tfb

METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = False

df0 = pd.read_csv('../../assets/data/monsters.csv')
print('Includes {} monsters in database'.format(len(df0.index.to_list())))

Includes 2144 monsters in database


In [2]:
# Fig. 1: plot the number of monsters by CR

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        barmode='stack',
        xaxis=dict(
            title_text='challenge rating',
            range=[0, 31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='monsters',
            #range=[0, 35],
            tick0=0, dtick=20,
            minor=dict(tick0=0, dtick=10),
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=1.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
            traceorder='normal',
        )
    )
)

def x_decoder(x):
    match x:
        case 0:
            return 0
        case 0.125:
            return 0.125
        case 0.25:
            return 0.28125
        case 0.5:
            return 0.5625
        case 1:
            return 1.125
        case _:
            return x

def w_decoder(x):
    match x:
        case 0:
            return 0.125
        case 0.125:
            return 0.125
        case 0.25:
            return 0.1875
        case 0.5:
            return 0.375
        case 1:
            return 0.75
        case _:
            return 1.0


for g in ['generic','legendary','unique']:
    dfC = df0[df0['CR'].gt(0) & df0['Category'].isin([g])].groupby(['CR']).count()
    x = [x_decoder(cr) for cr in dfC.index]
    w = [w_decoder(cr) for cr in dfC.index]
    cr = dfC.index.to_list()
    
    fig.add_trace(go.Bar(
        x=x,
        y=dfC['Category'],
        width=w,
        name=g,
        customdata=dfC.index,
        hovertemplate = '<b>' + g + '</b><br>'
            + 'CR %{customdata}<br>'
            + 'Monsters %{y:,.0f}'
            + '<extra></extra>'
    ))
    

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-monster-distribution-large')
    tfb.save_fig_html(fig, format='small', name='fig-monster-distribution-small')